In [1]:
import pandas as pd
import numpy as np
from functools import reduce
#from IPython.display import display
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

# Reading school data
dataEl_2012 = pd.read_csv('data/school/elementary_2012_13.csv', encoding='utf-8')
dataEl_2013 = pd.read_csv('data/school/elementary_2013_14.csv', encoding='utf-8')
dataEl_2014 = pd.read_csv('data/school/elementary_2014_15.csv', encoding='utf-8')
dataEl_2015 = pd.read_csv('data/school/elementary_2015_16.csv', encoding='utf-8')

# Elementary school appended dataframe

In [2]:
dataSchoolCom = dataEl_2015.append([dataEl_2014,dataEl_2013,dataEl_2012], sort=False)
#dataSchoolCom = dataSchoolCom.fillna(method = 'bfill', axis=0).fillna(0)
dataSchoolCom = dataSchoolCom.fillna(dataSchoolCom.mean())
dataSchoolCom.replace('ORISSA','ODISHA',inplace=True)
dataSchoolCom.replace('JAMMU AND KASHMIR','JAMMU & KASHMIR',inplace=True)
dataSchoolCom.replace('DELHI','NCT OF DELHI',inplace=True)
dataSchoolCom.replace('ANDAMAN AND NICOBAR ISLANDS','ANDAMAN & NICOBAR ISLANDS',inplace=True)
dataSchoolCom.replace('DAMAN AND DIU','DAMAN & DIU',inplace=True)
dataSchoolCom.replace('D AND N HAVELI','DADRA & NAGAR HAVELI',inplace=True)
dataSchoolCom.replace('A AND N ISLANDS','ANDAMAN & NICOBAR ISLANDS',inplace=True)
dataSchoolCom.replace('PONDICHERRY','PUDUCHERRY',inplace=True)
dataSchoolCom.head()

Unnamed: 0       STATE NAME DISTRICT NAME     YEAR  TOTAL POULATION  \
0           0  JAMMU & KASHMIR       KUPWARA  2015-16         875564.0   
1           1  JAMMU & KASHMIR      BARAMULA  2015-16        1015503.0   
2           2  JAMMU & KASHMIR      SRINAGAR  2015-16        1269751.0   
3           3  JAMMU & KASHMIR        BADGAM  2015-16         735753.0   
4           4  JAMMU & KASHMIR       PULWAMA  2015-16         570060.0   

   PERCENTAGE URBAN POPULATION  0-6 POPULATION  GROWTH RATE  SEX RATIO  \
0                        11.33       197001.90        34.62      843.0   
1                        17.19       161871.18        20.34      873.0   
2                        98.73       155798.45        23.56      879.0   
3                        11.74       150755.79        21.18      883.0   
4                        13.80        97651.28        29.18      913.0   

   PERCENTAGE SC POPULATION                  ...                    \
0                      0.12                  ...                     
1                      0.15                  ...                     
2                      0.09                  ...                     
3                      0.05                  ...                     
4                      0.07                  ...                     

   PRIMARY WITH UPPER PRIMARY (TRN_TCH_F2)  \
0                               419.463746   
1                               419.463746   
2                               419.463746   
3                               419.463746   
4                               419.463746   

   PRIMARY WITH UPPER PRIMARY SEC AND HIGHER SEC. (TRN_TCH_F3)  \
0                                          30.247734             
1                                          30.247734             
2                                          30.247734             
3                                          30.247734             
4                                          30.247734             

   UPPER PRIMARY ONLY (TRN_TCH_F4)  \
0                        72.720544   
1                        72.720544   
2                        72.720544   
3                        72.720544   
4                        72.720544   

   UPPER PRIMARY WITH SEC. AND HIGHER SEC. (TRN_TCH_F5)  \
0                                          93.587613      
1                                          93.587613      
2                                          93.587613      
3                                          93.587613      
4                                          93.587613      

   PRIMARY WITH UPPER PRIMARY SEC (TRN_TCH_F6)  \
0                                     15.41994   
1                                     15.41994   
2                                     15.41994   
3                                     15.41994   
4                                     15.41994   

    UPPER PRIMARY SEC (TRN_TCH_F7)  REGULAR TCH (PROF_TRN_TCH_R)  \
0                        37.042296                   8085.361027   
1                        37.042296                   8085.361027   
2                        37.042296                   8085.361027   
3                        37.042296                   8085.361027   
4                        37.042296                   8085.361027   

   CONTRACT TCH (PROF_TRN_TCH_P)  NO. OF DAYS INVOLVED  (DAYS_NONTCH)  \
0                     427.903323                            18.601208   
1                     427.903323                            18.601208   
2                     427.903323                            18.601208   
3                     427.903323                            18.601208   
4                     427.903323                            18.601208   

   NO. OF TEACHERS INVOLVED (TCH_NONTCH)  
0                             608.087613  
1                             608.087613  
2                             608.087613  
3                             608.087613  
4                             608.087613  

[5 rows x 1571 columns]

# School data methods

In [3]:
geography = pd.read_csv('data/geography.csv',encoding='UTF-8')
geo = geography[['geo_level','geo_code','name']]

# Calculating country data
def setCountryData(df,cols):
    for x in cols:
         df.loc[df['geo_level'] == 'country', x]= df.loc[df['geo_level'] == 'state', x].sum()
    return df;

def getGeoData(df):
    cols = list(df.columns.values)[2:]
    df = df.replace(',','',regex=True)
    df[cols] = df[cols].astype(float)
    df = df.rename(index=str, columns={'STATE NAME': 'name'})
    dfState = df.groupby(['name'])[cols].sum().reset_index()
   
    dfDis   = df.groupby(['DISTRICT NAME'])[cols].sum().reset_index()
    dfDis   = dfDis.rename(index=str, columns={'DISTRICT NAME': 'name'})
    dfDis['name'] = dfDis['name'].str.title()
    
    dfCom = pd.concat([dfState, dfDis])
    dfgeo = geo.merge(dfCom ,on=['name'],how='left')
    dfgeo = dfgeo.sort_values(by=['geo_level','name'])
    dfgeo = dfgeo.fillna(method = 'bfill', axis=0).fillna(0)
    dfgeoCountry = setCountryData(dfgeo,cols)
    return dfgeoCountry

# Manipulating school data
def getSchoolGeo(df,varname):
    df.columns = df.columns.str.split('(').str[0]
    df.columns = [varname + str(col) for col in df]
    statecols = varname + 'STATE NAME'
    discols   = varname + 'DISTRICT NAME'
    df      = df.rename(index=str, columns={statecols:'STATE NAME',discols:'DISTRICT NAME'})
    dfgeo   = getGeoData(df)
    return dfgeo

def getSchoolWazi(df1,df2):
    dfcom     = [df1,df2]
    dfcomgeo  = reduce(lambda left,right: pd.merge(left,right, on=['geo_level','geo_code','name']),dfcom)
    dfcomwazi = pd.melt(dfcomgeo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")
    dfcomwazi.insert(2,'geo_version','2011')
    dfcomwazi['total']  = dfcomwazi['total'].astype(int)
    return dfcomwazi

# Schools by category 2015

In [4]:
schooldata2013  =  dataSchoolCom.loc[dataSchoolCom['YEAR'] == '2013-14']
schooldata2015  =  dataSchoolCom.loc[dataSchoolCom['YEAR'] == '2015-16']
schoolgov2015df =  schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1G)','PRIMARY WITH UPPER PRIMARY (SCH2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3G)','UPPER PRIMARY ONLY (SCH4G)','UPPER PRIMARY WITH SEC./H.SEC (SCH5G)','PRIMARY WITH UPPER PRIMARY SEC (SCH6G)','UPPER PRIMARY WITH  SEC. (SCH7G)']]
schoolpvt2015df =  schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1P)','PRIMARY WITH UPPER PRIMARY (SCH2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3P)','UPPER PRIMARY ONLY (SCH4P)','UPPER PRIMARY WITH SEC./H.SEC (SCH5P)','PRIMARY WITH UPPER PRIMARY SEC (SCH6P)','UPPER PRIMARY WITH  SEC. (SCH7P)']]

Schoolsgov2015geo = getSchoolGeo(schoolgov2015df,'School_Government_2015_')
Schoolspvt2015geo = getSchoolGeo(schoolpvt2015df,'School_Private_2015_')

SchoolComwazi2015 = getSchoolWazi(Schoolsgov2015geo,Schoolspvt2015geo)
SchoolComwazi2015.insert(4,'schools',SchoolComwazi2015['schooltemp'].str.split('_').str[3].str.title())
SchoolComwazi2015.insert(5,'type',SchoolComwazi2015['schooltemp'].str.split('_').str[1])
#SchoolComwazi2015.insert(6,'year','2015')

School2015 = SchoolComwazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
School2015['geo_version'] = '2011'
School2015.head()
#School2015.to_csv('data/schools_by_type_2015.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type   total  
0  Government  708642  
1  Government  172607  
2  Government   12628  
3  Government  119634  
4  Government   19570

# Schools by category 2014

In [5]:
schooldata2014  =  dataSchoolCom.loc[dataSchoolCom['YEAR'] == '2014-15']

schoolgov2014df =  schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1G)','PRIMARY WITH UPPER PRIMARY (SCH2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3G)','UPPER PRIMARY ONLY (SCH4G)','UPPER PRIMARY WITH SEC./H.SEC (SCH5G)','PRIMARY WITH UPPER PRIMARY SEC (SCH6G)','UPPER PRIMARY WITH  SEC. (SCH7G)']]
schoolpvt2014df =  schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1P)','PRIMARY WITH UPPER PRIMARY (SCH2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3P)','UPPER PRIMARY ONLY (SCH4P)','UPPER PRIMARY WITH SEC./H.SEC (SCH5P)','PRIMARY WITH UPPER PRIMARY SEC (SCH6P)','UPPER PRIMARY WITH  SEC. (SCH7P)']]

Schoolsgov2014geo = getSchoolGeo(schoolgov2014df,'School_Government_2014_')
Schoolspvt2014geo = getSchoolGeo(schoolpvt2014df,'School_Private_2014_')

SchoolComwazi2014 = getSchoolWazi(Schoolsgov2014geo,Schoolspvt2014geo)
SchoolComwazi2014.insert(4,'schools',SchoolComwazi2014['schooltemp'].str.split('_').str[3].str.title())
SchoolComwazi2014.insert(5,'type',SchoolComwazi2014['schooltemp'].str.split('_').str[1])
#SchoolComwazi2014.insert(6,'year','2014')

School2014 = SchoolComwazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
School2014['geo_version'] = '2014'
School2014.head()
#School2014.to_csv('data/schools_by_type_2014.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2014                          Primary Only    
1   country       IN        2014            Primary With Upper Primary    
2   country       IN        2014  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2014                    Upper Primary Only    
4   country       IN        2014         Upper Primary With Sec./H.Sec    

         type   total  
0  Government  712767  
1  Government  172757  
2  Government    8716  
3  Government  120145  
4  Government   22081

# Schools by category combined test

In [8]:
schoolFinal = School2015.append(School2014)
schoolFinal.head()
#schoolFinal.to_csv('data/schooltest.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type   total  
0  Government  708642  
1  Government  172607  
2  Government   12628  
3  Government  119634  
4  Government   19570

# Schools by category 2013

In [28]:
schoolgov2013df =  schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1G)','PRIMARY WITH UPPER PRIMARY (SCH2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3G)','UPPER PRIMARY ONLY (SCH4G)','UPPER PRIMARY WITH SEC./H.SEC (SCH5G)','PRIMARY WITH UPPER PRIMARY SEC (SCH6G)','UPPER PRIMARY WITH  SEC. (SCH7G)']]
schoolpvt2013df =  schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCH1P)','PRIMARY WITH UPPER PRIMARY (SCH2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3P)','UPPER PRIMARY ONLY (SCH4P)','UPPER PRIMARY WITH SEC./H.SEC (SCH5P)','PRIMARY WITH UPPER PRIMARY SEC (SCH6P)','UPPER PRIMARY WITH  SEC. (SCH7P)']]

Schoolsgov2013geo = getSchoolGeo(schoolgov2013df,'School_Government_2013_')
Schoolspvt2013geo = getSchoolGeo(schoolpvt2013df,'School_Private_2013_')

SchoolComwazi2013 = getSchoolWazi(Schoolsgov2013geo,Schoolspvt2013geo)
SchoolComwazi2013.insert(4,'schools',SchoolComwazi2013['schooltemp'].str.split('_').str[3].str.title())
SchoolComwazi2013.insert(5,'type',SchoolComwazi2013['schooltemp'].str.split('_').str[1])
SchoolComwazi2013.insert(6,'year','2013')

School2013 = SchoolComwazi2013.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
School2013.head()
#School2013.to_csv('data/schools_by_type_2013.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year   total  
0  Government  2013  674222  
1  Government  2013  163902  
2  Government  2013   10123  
3  Government  2013  113683  
4  Government  2013   19752

# Schools by gender boys/girls 2015

In [6]:
schoolboys2015df  = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHBOY1)','PRIMARY WITH UPPER PRIMARY (SCHBOY2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHBOY3)','UPPER PRIMARY ONLY (SCHBOY4)','UPPER PRIMARY WITH SEC./H.SEC (SCHBOY5)','PRIMARY WITH UPPER PRIMARY SEC (SCHBOY6)','UPPER PRIMARY WITH  SEC. (SCHBOY7)']]
schoolgirls2015df = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHGIR1)','PRIMARY WITH UPPER PRIMARY (SCHGIR2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHGIR3)','UPPER PRIMARY ONLY (SCHGIR4)','UPPER PRIMARY WITH SEC./H.SEC (SCHGIR5)','PRIMARY WITH UPPER PRIMARY SEC (SCHGIR6)','UPPER PRIMARY WITH  SEC. (SCHGIR7)']]

Schoolboys2015geo  = getSchoolGeo(schoolboys2015df,'School_Boys_2015_')
Schoolgirls2015geo = getSchoolGeo(schoolgirls2015df,'School_Girls_2015_')

Schoolbygenwazi2015 =  getSchoolWazi(Schoolboys2015geo,Schoolgirls2015geo)
Schoolbygenwazi2015.insert(4,'schools',Schoolbygenwazi2015['schooltemp'].str.split('_').str[3].str.title())
Schoolbygenwazi2015.insert(5,'gender',Schoolbygenwazi2015['schooltemp'].str.split('_').str[1])
Schoolbygenwazi2015.insert(6,'year','2015')
Schoolbygenwazi2015df= Schoolbygenwazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Schoolbygenwazi2015df.head()
#Schoolbygenwazi2015df.to_csv('data/schools_by_gender_2015.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  gender  year   total  
0   Boys  2015  832505  
1   Boys  2015  278088  
2   Boys  2015   40088  
3   Boys  2015  140800  
4   Boys  2015   31669

# Schools by gender boys/girls 2014

In [7]:
schoolboys2014df  = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHBOY1)','PRIMARY WITH UPPER PRIMARY (SCHBOY2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHBOY3)','UPPER PRIMARY ONLY (SCHBOY4)','UPPER PRIMARY WITH SEC./H.SEC (SCHBOY5)','PRIMARY WITH UPPER PRIMARY SEC (SCHBOY6)','UPPER PRIMARY WITH  SEC. (SCHBOY7)']]
schoolgirls2014df = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHGIR1)','PRIMARY WITH UPPER PRIMARY (SCHGIR2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHGIR3)','UPPER PRIMARY ONLY (SCHGIR4)','UPPER PRIMARY WITH SEC./H.SEC (SCHGIR5)','PRIMARY WITH UPPER PRIMARY SEC (SCHGIR6)','UPPER PRIMARY WITH  SEC. (SCHGIR7)']]

Schoolboys2014geo  = getSchoolGeo(schoolboys2014df,'School_Boys_2014_')
Schoolgirls2014geo = getSchoolGeo(schoolgirls2014df,'School_Girls_2014_')

Schoolbygenwazi2014 =  getSchoolWazi(Schoolboys2014geo,Schoolgirls2014geo)
Schoolbygenwazi2014.insert(4,'schools',Schoolbygenwazi2014['schooltemp'].str.split('_').str[3].str.title())
Schoolbygenwazi2014.insert(5,'gender',Schoolbygenwazi2014['schooltemp'].str.split('_').str[1])
Schoolbygenwazi2014.insert(6,'year','2014')
Schoolbygenwazi2014df= Schoolbygenwazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Schoolbygenwazi2014df.head()
#Schoolbygenwazi2014df.to_csv('data/schools_by_gender_2014.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  gender  year   total  
0   Boys  2014  838451  
1   Boys  2014  273585  
2   Boys  2014   36367  
3   Boys  2014  140372  
4   Boys  2014   34439

# Schools by gender boys/girls 2013

In [31]:
schoolboys2013df  = schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHBOY1)','PRIMARY WITH UPPER PRIMARY (SCHBOY2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHBOY3)','UPPER PRIMARY ONLY (SCHBOY4)','UPPER PRIMARY WITH SEC./H.SEC (SCHBOY5)','PRIMARY WITH UPPER PRIMARY SEC (SCHBOY6)','UPPER PRIMARY WITH  SEC. (SCHBOY7)']]
schoolgirls2013df = schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SCHGIR1)','PRIMARY WITH UPPER PRIMARY (SCHGIR2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCHGIR3)','UPPER PRIMARY ONLY (SCHGIR4)','UPPER PRIMARY WITH SEC./H.SEC (SCHGIR5)','PRIMARY WITH UPPER PRIMARY SEC (SCHGIR6)','UPPER PRIMARY WITH  SEC. (SCHGIR7)']]

Schoolboys2013geo  = getSchoolGeo(schoolboys2013df,'School_Boys_2013_')
Schoolgirls2013geo = getSchoolGeo(schoolgirls2013df,'School_Girls_2013_')

Schoolbygenwazi2013 =  getSchoolWazi(Schoolboys2013geo,Schoolgirls2013geo)
Schoolbygenwazi2013.insert(4,'schools',Schoolbygenwazi2013['schooltemp'].str.split('_').str[3].str.title())
Schoolbygenwazi2013.insert(5,'gender',Schoolbygenwazi2013['schooltemp'].str.split('_').str[1])
Schoolbygenwazi2013.insert(6,'year','2013')
Schoolbygenwazi2013df= Schoolbygenwazi2013.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Schoolbygenwazi2013df.head()
#Schoolbygenwazi2013df.to_csv('data/schools_by_gender_2013.csv', index=False)

geo_level geo_code geo_version                                schools  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  gender  year   total  
0   Boys  2013  792586  
1   Boys  2013  261754  
2   Boys  2013   36258  
3   Boys  2013  133316  
4   Boys  2013   31350

# Students enrolment by school category 2015

In [8]:
studentsenrolgov2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1G)','PRIMARY WITH UPPER PRIMARY (ENR2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3G)','UPPER PRIMARY ONLY (ENR4G)','UPPER PRIMARY WITH SEC./H.SEC (ENR5G)','PRIMARY WITH UPPER PRIMARY SEC (ENR6G)','UPPER PRIMARY WITH  SEC. (ENR7G)']]
studentsenrolpvt2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1P)','PRIMARY WITH UPPER PRIMARY (ENR2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3P)','UPPER PRIMARY ONLY (ENR4P)','UPPER PRIMARY WITH SEC./H.SEC (ENR5P)','PRIMARY WITH UPPER PRIMARY SEC (ENR6P)','UPPER PRIMARY WITH  SEC. (ENR7P)']]

Studentsengov2015geo  = getSchoolGeo(studentsenrolgov2015,'Studentsenrol_Government_2015_')
Studentsenpvt2015geo  = getSchoolGeo(studentsenrolpvt2015,'Studentsenrol_Private_2015_')

Studentsenwazi2015    =  getSchoolWazi(Studentsengov2015geo,Studentsenpvt2015geo)
Studentsenwazi2015.insert(4,'studentsenrol',Studentsenwazi2015['schooltemp'].str.split('_').str[3].str.title())
Studentsenwazi2015.insert(5,'type',Studentsenwazi2015['schooltemp'].str.split('_').str[1])
Studentsenwazi2015.insert(6,'year','2015')
Studentsenwazi2015df= Studentsenwazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Studentsenwazi2015df.head()
#Studentsenwazi2015df.to_csv('data/studentsenrol_type_2015.csv', index=False)

geo_level geo_code geo_version                          studentsenrol  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year     total  
0  Government  2015  51596535  
1  Government  2015  35644165  
2  Government  2015   3894845  
3  Government  2015  10808578  
4  Government  2015   6504595

# Students enrolment by school category 2014

In [9]:
studentsenrolgov2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1G)','PRIMARY WITH UPPER PRIMARY (ENR2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3G)','UPPER PRIMARY ONLY (ENR4G)','UPPER PRIMARY WITH SEC./H.SEC (ENR5G)','PRIMARY WITH UPPER PRIMARY SEC (ENR6G)','UPPER PRIMARY WITH  SEC. (ENR7G)']]
studentsenrolpvt2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1P)','PRIMARY WITH UPPER PRIMARY (ENR2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3P)','UPPER PRIMARY ONLY (ENR4P)','UPPER PRIMARY WITH SEC./H.SEC (ENR5P)','PRIMARY WITH UPPER PRIMARY SEC (ENR6P)','UPPER PRIMARY WITH  SEC. (ENR7P)']]

Studentsengov2014geo  = getSchoolGeo(studentsenrolgov2014,'Studentsenrol_Government_2014_')
Studentsenpvt2014geo  = getSchoolGeo(studentsenrolpvt2014,'Studentsenrol_Private_2014_')

Studentsenwazi2014    =  getSchoolWazi(Studentsengov2014geo,Studentsenpvt2014geo)
Studentsenwazi2014.insert(4,'studentsenrol',Studentsenwazi2014['schooltemp'].str.split('_').str[3].str.title())
Studentsenwazi2014.insert(5,'type',Studentsenwazi2014['schooltemp'].str.split('_').str[1])
Studentsenwazi2014.insert(6,'year','2014')
Studentsenwazi2014df= Studentsenwazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Studentsenwazi2014df.head()
#Studentsenwazi2014df.to_csv('data/studentsenrol_type_2014.csv', index=False)

geo_level geo_code geo_version                          studentsenrol  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year     total  
0  Government  2014  53616697  
1  Government  2014  35480727  
2  Government  2014   3039479  
3  Government  2014  11252734  
4  Government  2014   6914053

# Students enrolment by school category 2013

In [34]:
studentsenrolgov2013 = schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1G)','PRIMARY WITH UPPER PRIMARY (ENR2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3G)','UPPER PRIMARY ONLY (ENR4G)','UPPER PRIMARY WITH SEC./H.SEC (ENR5G)','PRIMARY WITH UPPER PRIMARY SEC (ENR6G)','UPPER PRIMARY WITH  SEC. (ENR7G)']]
studentsenrolpvt2013 = schooldata2013[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENR1P)','PRIMARY WITH UPPER PRIMARY (ENR2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENR3P)','UPPER PRIMARY ONLY (ENR4P)','UPPER PRIMARY WITH SEC./H.SEC (ENR5P)','PRIMARY WITH UPPER PRIMARY SEC (ENR6P)','UPPER PRIMARY WITH  SEC. (ENR7P)']]

Studentsengov2013geo  = getSchoolGeo(studentsenrolgov2013,'Studentsenrol_Government_2013_')
Studentsenpvt2013geo  = getSchoolGeo(studentsenrolpvt2013,'Studentsenrol_Private_2013_')

Studentsenwazi2013    =  getSchoolWazi(Studentsengov2013geo,Studentsenpvt2013geo)
Studentsenwazi2013.insert(4,'studentsenrol',Studentsenwazi2013['schooltemp'].str.split('_').str[3].str.title())
Studentsenwazi2013.insert(5,'type',Studentsenwazi2013['schooltemp'].str.split('_').str[1])
Studentsenwazi2013.insert(6,'year','2013')
Studentsenwazi2013df= Studentsenwazi2013.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
Studentsenwazi2013df.head()
#Studentsenwazi2013df.to_csv('data/studentsenrol_type_2013.csv', index=False)

geo_level geo_code geo_version                          studentsenrol  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year     total  
0  Government  2013  49897622  
1  Government  2013  33763587  
2  Government  2013   3289365  
3  Government  2013  10457760  
4  Government  2013   6363931

# Teachers by school category 2015

In [10]:
teachersgov2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCH1G)','PRIMARY WITH UPPER PRIMARY (TCH2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCH3G)','UPPER PRIMARY ONLY (TCH4G)','UPPER PRIMARY WITH SEC./H.SEC (TCH5G)','PRIMARY WITH UPPER PRIMARY SEC (TCH6G)','UPPER PRIMARY WITH  SEC. (TCH7G)']]
teacherspvt2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCH1P)','PRIMARY WITH UPPER PRIMARY (TCH2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCH3P)','UPPER PRIMARY ONLY (TCH4P)','UPPER PRIMARY WITH SEC./H.SEC (TCH5P)','PRIMARY WITH UPPER PRIMARY SEC (TCH6P)','UPPER PRIMARY WITH  SEC. (TCH7P)']]

teachersgov2015geo  = getSchoolGeo(teachersgov2015,'Teachers_Government_2015_')
teacherspvt2015geo  = getSchoolGeo(teacherspvt2015,'Teachers_Private_2015_')

teacherswazi2015    =  getSchoolWazi(teachersgov2015geo,teacherspvt2015geo)
teacherswazi2015.insert(4,'teachers',teacherswazi2015['schooltemp'].str.split('_').str[3].str.title())
teacherswazi2015.insert(5,'type',teacherswazi2015['schooltemp'].str.split('_').str[1])
teacherswazi2015.insert(6,'year','2015')
teacherswazi2015df= teacherswazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
teacherswazi2015df.head()
#teacherswazi2015df.to_csv('data/teachers_type_2015.csv', index=False)

geo_level geo_code geo_version                               teachers  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year    total  
0  Government  2015  1941226  
1  Government  2015  1114863  
2  Government  2015   229340  
3  Government  2015   437705  
4  Government  2015   456013

# Teachers by school category 2014

In [11]:
teachersgov2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCH1G)','PRIMARY WITH UPPER PRIMARY (TCH2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCH3G)','UPPER PRIMARY ONLY (TCH4G)','UPPER PRIMARY WITH SEC./H.SEC (TCH5G)','PRIMARY WITH UPPER PRIMARY SEC (TCH6G)','UPPER PRIMARY WITH  SEC. (TCH7G)']]
teacherspvt2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCH1P)','PRIMARY WITH UPPER PRIMARY (TCH2P)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCH3P)','UPPER PRIMARY ONLY (TCH4P)','UPPER PRIMARY WITH SEC./H.SEC (TCH5P)','PRIMARY WITH UPPER PRIMARY SEC (TCH6P)','UPPER PRIMARY WITH  SEC. (TCH7P)']]

teachersgov2014geo  = getSchoolGeo(teachersgov2014,'Teachers_Government_2014_')
teacherspvt2014geo  = getSchoolGeo(teacherspvt2014,'Teachers_Private_2014_')

teacherswazi2014    =  getSchoolWazi(teachersgov2014geo,teacherspvt2014geo)
teacherswazi2014.insert(4,'teachers',teacherswazi2014['schooltemp'].str.split('_').str[3].str.title())
teacherswazi2014.insert(5,'type',teacherswazi2014['schooltemp'].str.split('_').str[1])
teacherswazi2014.insert(6,'year','2014')
teacherswazi2014df= teacherswazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
teacherswazi2014df.head()
#teacherswazi2014df.to_csv('data/teachers_type_2014.csv', index=False)

geo_level geo_code geo_version                               teachers  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

         type  year    total  
0  Government  2014  1988318  
1  Government  2014  1125943  
2  Government  2014   181965  
3  Government  2014   424646  
4  Government  2014   482758

# Teachers by gender 2015

In [12]:
teachersfemale2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCHF1)','PRIMARY WITH UPPER PRIMARY (TCHF2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCHF3)','UPPER PRIMARY ONLY (TCHF4)','UPPER PRIMARY WITH SEC./H.SEC (TCHF5)','PRIMARY WITH UPPER PRIMARY SEC (TCHF6)','UPPER PRIMARY WITH  SEC. (TCHF7)']]
teachersmale2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCHM1)','PRIMARY WITH UPPER PRIMARY (TCHM2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCHM3)','UPPER PRIMARY ONLY (TCHM4)','UPPER PRIMARY WITH SEC./H.SEC (TCHM5)','PRIMARY WITH UPPER PRIMARY SEC (TCHM6)','UPPER PRIMARY WITH  SEC. (TCHM7)']]

teachersfemale2015geo  = getSchoolGeo(teachersfemale2015,'Teachers_Female_2015_')
teachersmale2015geo  = getSchoolGeo(teachersmale2015,'Teachers_male_2015_')

teachersgenderwazi2015    =  getSchoolWazi(teachersfemale2015geo,teachersmale2015geo)
teachersgenderwazi2015.insert(4,'teachers',teachersgenderwazi2015['schooltemp'].str.split('_').str[3].str.title())
teachersgenderwazi2015.insert(5,'gender',teachersgenderwazi2015['schooltemp'].str.split('_').str[1])
teachersgenderwazi2015.insert(6,'year','2015')
teachersgenderwazi2015df= teachersgenderwazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
teachersgenderwazi2015df.head()
#teachersgenderwazi2015df.to_csv('data/teachers_gender_2015.csv', index=False)

geo_level geo_code geo_version                               teachers  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

   gender  year    total  
0  Female  2015  1231720  
1  Female  2015  1002520  
2  Female  2015   545545  
3  Female  2015   196620  
4  Female  2015   312336

# Teachers by gender 2014

In [13]:
teachersfemale2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCHF1)','PRIMARY WITH UPPER PRIMARY (TCHF2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCHF3)','UPPER PRIMARY ONLY (TCHF4)','UPPER PRIMARY WITH SEC./H.SEC (TCHF5)','PRIMARY WITH UPPER PRIMARY SEC (TCHF6)','UPPER PRIMARY WITH  SEC. (TCHF7)']]
teachersmale2014   = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TCHM1)','PRIMARY WITH UPPER PRIMARY (TCHM2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TCHM3)','UPPER PRIMARY ONLY (TCHM4)','UPPER PRIMARY WITH SEC./H.SEC (TCHM5)','PRIMARY WITH UPPER PRIMARY SEC (TCHM6)','UPPER PRIMARY WITH  SEC. (TCHM7)']]

teachersfemale2014geo  = getSchoolGeo(teachersfemale2014,'Teachers_Female_2014_')
teachersmale2014geo  = getSchoolGeo(teachersmale2014,'Teachers_male_2014_')

teachersgenderwazi2014    =  getSchoolWazi(teachersfemale2014geo,teachersmale2014geo)
teachersgenderwazi2014.insert(4,'teachers',teachersgenderwazi2014['schooltemp'].str.split('_').str[3].str.title())
teachersgenderwazi2014.insert(5,'gender',teachersgenderwazi2014['schooltemp'].str.split('_').str[1])
teachersgenderwazi2014.insert(6,'year','2014')
teachersgenderwazi2014df= teachersgenderwazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
teachersgenderwazi2014df.head()
#teachersgenderwazi2014df.to_csv('data/teachers_gender_2014.csv', index=False)

geo_level geo_code geo_version                               teachers  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

   gender  year    total  
0  Female  2014  1260992  
1  Female  2014   982665  
2  Female  2014   528269  
3  Female  2014   186429  
4  Female  2014   334104

# Schools with toilet facilities 2015

In [14]:
toiletgirls2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SGTOIL1)','PRIMARY WITH UPPER PRIMARY (SGTOIL2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SGTOIL3)','UPPER PRIMARY ONLY (SGTOIL4)','UPPER PRIMARY WITH SEC./H.SEC (SGTOIL5)','PRIMARY WITH UPPER PRIMARY SEC (SGTOIL6)','UPPER PRIMARY WITH  SEC. (SGTOIL7)']]
toiletboys2015  = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SBTOIL1)','PRIMARY WITH UPPER PRIMARY (SBTOIL2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SBTOIL3)','UPPER PRIMARY ONLY (SBTOIL4)','UPPER PRIMARY WITH SEC./H.SEC (SBTOIL5)','PRIMARY WITH UPPER PRIMARY SEC (SBTOIL6)','UPPER PRIMARY WITH  SEC. (SBTOIL7)']]

toiletgirls2015geo  = getSchoolGeo(toiletgirls2015,'Toilet_girls_2015_')
toiletboys2015geo   = getSchoolGeo(toiletboys2015,'Toilet_boys_2015_')

toiletwazi2015      =  getSchoolWazi(toiletgirls2015geo,toiletboys2015geo)
toiletwazi2015.insert(4,'schooltoilet',toiletwazi2015['schooltemp'].str.split('_').str[3].str.title())
toiletwazi2015.insert(5,'gender',toiletwazi2015['schooltemp'].str.split('_').str[1])
toiletwazi2015.insert(6,'year','2015')
toiletwazi2015df= toiletwazi2015.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
toiletwazi2015df.head()
#toiletwazi2015df.to_csv('data/schooltoilet_by_gender_2015.csv', index=False)

geo_level geo_code geo_version                           schooltoilet  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  gender  year   total  
0  girls  2015  808547  
1  girls  2015  276820  
2  girls  2015   40597  
3  girls  2015  140166  
4  girls  2015   33385

# Schools with toilet facilities 2014

In [15]:
toiletgirls2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SGTOIL1)','PRIMARY WITH UPPER PRIMARY (SGTOIL2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SGTOIL3)','UPPER PRIMARY ONLY (SGTOIL4)','UPPER PRIMARY WITH SEC./H.SEC (SGTOIL5)','PRIMARY WITH UPPER PRIMARY SEC (SGTOIL6)','UPPER PRIMARY WITH  SEC. (SGTOIL7)']]
toiletboys2014  = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (SBTOIL1)','PRIMARY WITH UPPER PRIMARY (SBTOIL2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SBTOIL3)','UPPER PRIMARY ONLY (SBTOIL4)','UPPER PRIMARY WITH SEC./H.SEC (SBTOIL5)','PRIMARY WITH UPPER PRIMARY SEC (SBTOIL6)','UPPER PRIMARY WITH  SEC. (SBTOIL7)']]

toiletgirls2014geo  = getSchoolGeo(toiletgirls2014,'Toilet_girls_2014_')
toiletboys2014geo   = getSchoolGeo(toiletboys2014,'Toilet_boys_2014_')

toiletwazi2014      =  getSchoolWazi(toiletgirls2014geo,toiletboys2014geo)
toiletwazi2014.insert(4,'schooltoilet',toiletwazi2014['schooltemp'].str.split('_').str[3].str.title())
toiletwazi2014.insert(5,'gender',toiletwazi2014['schooltemp'].str.split('_').str[1])
toiletwazi2014.insert(6,'year','2014')
toiletwazi2014df= toiletwazi2014.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
toiletwazi2014df.head()
#toiletwazi2014df.to_csv('data/schooltoilet_by_gender_2014.csv', index=False)

geo_level geo_code geo_version                           schooltoilet  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  gender  year   total  
0  girls  2014  700661  
1  girls  2014  257416  
2  girls  2014   36614  
3  girls  2014  124639  
4  girls  2014   35780

# Schools with basic facilities 2015

In [16]:
schoolbfac2015     = schooldata2015[['STATE NAME','DISTRICT NAME','TOTAL (SPLAYTOT)','TOTAL (SBNDRTOT)','TOTAL (SGTOILTOT)','TOTAL (SBTOILTOT)','TOTAL (SWATTOT)','TOTAL (SELETOT)','TOTAL (SCOMPTOT)']]
schoolbfac2015     = schoolbfac2015.rename(index=str, columns={'TOTAL (SPLAYTOT)': 'Playground (total)','TOTAL (SBNDRTOT)':'Boundarywall (Total)','TOTAL (SGTOILTOT)':'Girls toilet (total)','TOTAL (SBTOILTOT)':'Boys toilet (total)','TOTAL (SWATTOT)':'Drinking water (total)','TOTAL (SELETOT)':'Electricity (total)','TOTAL (SCOMPTOT)':'Computer (total)'})
schoolbfac2015geo  = getSchoolGeo(schoolbfac2015,'basic_facilities_2015_')
schoolbfac2015wazi = pd.melt(schoolbfac2015geo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")
schoolbfac2015wazi.insert(2,'geo_version','2011')
schoolbfac2015wazi['total']  = schoolbfac2015wazi['total'].astype(int)

schoolbfac2015wazi.insert(4,'schoolfacilities',schoolbfac2015wazi['schooltemp'].str.split('_').str[3].str.title())
schoolbfac2015wazi.insert(5,'year','2015')
schoolbfac2015wazidf = schoolbfac2015wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
schoolbfac2015wazidf.head()
#schoolbfac2015wazidf.to_csv('data/schoolfacilities_2015.csv', index=False)

geo_level geo_code geo_version schoolfacilities  year    total
0   country       IN        2011      Playground   2015   877416
1   country       IN        2011    Boundarywall   2015   924374
2   country       IN        2011    Girls Toilet   2015  1398928
3   country       IN        2011     Boys Toilet   2015  1377924
4   country       IN        2011  Drinking Water   2015  1401890

# Schools with basic facilities 2014

In [17]:
schoolbfac2014     = schooldata2014[['STATE NAME','DISTRICT NAME','TOTAL (SPLAYTOT)','TOTAL (SBNDRTOT)','TOTAL (SGTOILTOT)','TOTAL (SBTOILTOT)','TOTAL (SWATTOT)','TOTAL (SELETOT)','TOTAL (SCOMPTOT)']]
schoolbfac2014     = schoolbfac2014.rename(index=str, columns={'TOTAL (SPLAYTOT)': 'Playground (total)','TOTAL (SBNDRTOT)':'Boundarywall (Total)','TOTAL (SGTOILTOT)':'Girls toilet (total)','TOTAL (SBTOILTOT)':'Boys toilet (total)','TOTAL (SWATTOT)':'Drinking water (total)','TOTAL (SELETOT)':'Electricity (total)','TOTAL (SCOMPTOT)':'Computer (total)'})
schoolbfac2014geo  = getSchoolGeo(schoolbfac2014,'basic_facilities_2014_')
schoolbfac2014wazi = pd.melt(schoolbfac2014geo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")
schoolbfac2014wazi.insert(2,'geo_version','2011')
schoolbfac2014wazi['total']  = schoolbfac2014wazi['total'].astype(int)

schoolbfac2014wazi.insert(4,'schoolfacilities',schoolbfac2014wazi['schooltemp'].str.split('_').str[3].str.title())
schoolbfac2014wazi.insert(5,'year','2014')
schoolbfac2014wazidf = schoolbfac2014wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
schoolbfac2014wazidf.head()
#schoolbfac2014wazidf.to_csv('data/schoolfacilities_2014.csv', index=False)

geo_level geo_code geo_version schoolfacilities  year    total
0   country       IN        2011      Playground   2014   864304
1   country       IN        2011    Boundarywall   2014   914976
2   country       IN        2011    Girls Toilet   2014  1244023
3   country       IN        2011     Boys Toilet   2014  1350424
4   country       IN        2011  Drinking Water   2014  1388678

# Girls enrolment 2015

In [18]:
girlsenrol2015     = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENRG1)','PRIMARY WITH UPPER PRIMARY (ENRG2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENRG3)','UPPER PRIMARY ONLY (ENRG4)','UPPER PRIMARY WITH SEC./H.SEC (ENRG5)','PRIMARY WITH UPPER PRIMARY SEC (ENRG6)','UPPER PRIMARY WITH  SEC. (ENRG7)']]
girlsenrol2015geo  = getSchoolGeo(girlsenrol2015,'girlsenrolment_2015_')

girlsenrol2015wazi = pd.melt(girlsenrol2015geo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")
girlsenrol2015wazi.insert(2,'geo_version','2011')
girlsenrol2015wazi['total']  = girlsenrol2015wazi['total'].astype(int)
girlsenrol2015wazi.insert(4,'girlsenrolment',girlsenrol2015wazi['schooltemp'].str.split('_').str[2].str.title())
girlsenrol2015wazi.insert(5,'year','2015')

girlsenrol2015wazidf = girlsenrol2015wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
girlsenrol2015wazidf.head()
#girlsenrol2015wazidf.to_csv('data/girlsenrolment_2015.csv', index=False)

geo_level geo_code geo_version                         girlsenrolment  year  \
0   country       IN        2011                          Primary Only   2015   
1   country       IN        2011            Primary With Upper Primary   2015   
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec   2015   
3   country       IN        2011                    Upper Primary Only   2015   
4   country       IN        2011         Upper Primary With Sec./H.Sec   2015   

      total  
0  34392532  
1  28374986  
2   7952191  
3   7540610  
4   5573166

# Girls enrolment 2014

In [19]:
girlsenrol2014     = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (ENRG1)','PRIMARY WITH UPPER PRIMARY (ENRG2)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (ENRG3)','UPPER PRIMARY ONLY (ENRG4)','UPPER PRIMARY WITH SEC./H.SEC (ENRG5)','PRIMARY WITH UPPER PRIMARY SEC (ENRG6)','UPPER PRIMARY WITH  SEC. (ENRG7)']]
girlsenrol2014geo  = getSchoolGeo(girlsenrol2014,'girlsenrolment_2014_')

girlsenrol2014wazi = pd.melt(girlsenrol2014geo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")
girlsenrol2014wazi.insert(2,'geo_version','2011')
girlsenrol2014wazi['total']  = girlsenrol2014wazi['total'].astype(int)
girlsenrol2014wazi.insert(4,'girlsenrolment',girlsenrol2014wazi['schooltemp'].str.split('_').str[2].str.title())
girlsenrol2014wazi.insert(5,'year','2014')

girlsenrol2014wazidf = girlsenrol2014wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
girlsenrol2014wazidf.head()
#girlsenrol2014wazidf.to_csv('data/girlsenrolment_2014.csv', index=False)

geo_level geo_code geo_version                         girlsenrolment  year  \
0   country       IN        2011                          Primary Only   2014   
1   country       IN        2011            Primary With Upper Primary   2014   
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec   2014   
3   country       IN        2011                    Upper Primary Only   2014   
4   country       IN        2011         Upper Primary With Sec./H.Sec   2014   

      total  
0  35927186  
1  28058895  
2   7500607  
3   7783013  
4   5846002

# Classrooms conditions 2015

In [20]:
classroomConGood2015   = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLGD1G)','PRIMARY WITH UPPER PRIMARY (TOTCLGD2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLGD3G)','UPPER PRIMARY ONLY (TOTCLGD4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLGD5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLGD6G)','UPPER PRIMARY WITH  SEC. (TOTCLGD7G)']]
classroomConMinRep2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLMI1G)','PRIMARY WITH UPPER PRIMARY (TOTCLMI2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLMI3G)','UPPER PRIMARY ONLY (TOTCLMI4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLMI5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLMI6G)','UPPER PRIMARY WITH  SEC. (TOTCLMI7G)']]
classroomConMajRep2015 = schooldata2015[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLMJ1G)','PRIMARY WITH UPPER PRIMARY (TOTCLMJ2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLMJ3G)','UPPER PRIMARY ONLY (TOTCLMJ4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLMJ5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLMJ6G)','UPPER PRIMARY WITH  SEC. (TOTCLMJ7G)']]

classroomConGood2015geo   = getSchoolGeo(classroomConGood2015,'classroom_Good_2015_')
classroomConMinRep2015geo = getSchoolGeo(classroomConMinRep2015,'classroom_Need minor repair_2015_')
classroomConMajRep2015geo = getSchoolGeo(classroomConMajRep2015,'classroom_Need major repair_2015_')

classroomCon2015com  = [classroomConGood2015geo,classroomConMinRep2015geo,classroomConMajRep2015geo]
classroomCon2015comgeo  = reduce(lambda left,right: pd.merge(left,right, on=['geo_level','geo_code','name']),classroomCon2015com)

classroomCon2015wazi = pd.melt(classroomCon2015comgeo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")

classroomCon2015wazi.insert(2,'geo_version','2011')
classroomCon2015wazi['total']  = classroomCon2015wazi['total'].astype(int)
classroomCon2015wazi.insert(4,'classroom',classroomCon2015wazi['schooltemp'].str.split('_').str[3].str.title())
classroomCon2015wazi.insert(5,'conditions',classroomCon2015wazi['schooltemp'].str.split('_').str[1].str.title())
classroomCon2015wazi.insert(6,'year','2015')

classroomCon2015wazidf = classroomCon2015wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
classroomCon2015wazidf.head()
#classroomCon2015wazidf.to_csv('data/classroom_conditions_2015.csv', index=False)

geo_level geo_code geo_version                              classroom  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  conditions  year    total  
0       Good  2015  2359171  
1       Good  2015  1821141  
2       Good  2015   495727  
3       Good  2015   476263  
4       Good  2015   226718

# Classrooms conditions 2014

In [21]:
classroomConGood2014   = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLGD1G)','PRIMARY WITH UPPER PRIMARY (TOTCLGD2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLGD3G)','UPPER PRIMARY ONLY (TOTCLGD4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLGD5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLGD6G)','UPPER PRIMARY WITH  SEC. (TOTCLGD7G)']]
classroomConMinRep2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLMI1G)','PRIMARY WITH UPPER PRIMARY (TOTCLMI2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLMI3G)','UPPER PRIMARY ONLY (TOTCLMI4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLMI5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLMI6G)','UPPER PRIMARY WITH  SEC. (TOTCLMI7G)']]
classroomConMajRep2014 = schooldata2014[['STATE NAME','DISTRICT NAME','PRIMARY ONLY (TOTCLMJ1G)','PRIMARY WITH UPPER PRIMARY (TOTCLMJ2G)','PRIMARY WITH UPPER PRIMARY SEC/H.SEC (TOTCLMJ3G)','UPPER PRIMARY ONLY (TOTCLMJ4G)','UPPER PRIMARY WITH SEC./H.SEC (TOTCLMJ5G)','PRIMARY WITH UPPER PRIMARY SEC (TOTCLMJ6G)','UPPER PRIMARY WITH  SEC. (TOTCLMJ7G)']]

classroomConGood2014geo   = getSchoolGeo(classroomConGood2014,'classroom_Good_2014_')
classroomConMinRep2014geo = getSchoolGeo(classroomConMinRep2014,'classroom_Need minor repair_2014_')
classroomConMajRep2014geo = getSchoolGeo(classroomConMajRep2014,'classroom_Need major repair_2014_')

classroomCon2014com  = [classroomConGood2014geo,classroomConMinRep2014geo,classroomConMajRep2014geo]
classroomCon2014comgeo  = reduce(lambda left,right: pd.merge(left,right, on=['geo_level','geo_code','name']),classroomCon2014com)

classroomCon2014wazi = pd.melt(classroomCon2014comgeo, id_vars=['geo_level', 'geo_code','name'], var_name='schooltemp', value_name="total")

classroomCon2014wazi.insert(2,'geo_version','2011')
classroomCon2014wazi['total']  = classroomCon2014wazi['total'].astype(int)
classroomCon2014wazi.insert(4,'classroom',classroomCon2014wazi['schooltemp'].str.split('_').str[3].str.title())
classroomCon2014wazi.insert(5,'conditions',classroomCon2014wazi['schooltemp'].str.split('_').str[1].str.title())
classroomCon2014wazi.insert(6,'year','2014')

classroomCon2014wazidf = classroomCon2014wazi.sort_values(by=['geo_level','name']).reset_index().drop(['index'],axis=1).drop(['name'],axis=1).drop(['schooltemp'],axis=1)
classroomCon2014wazidf.head()
#classroomCon2014wazidf.to_csv('data/classroom_conditions_2014.csv', index=False)

geo_level geo_code geo_version                              classroom  \
0   country       IN        2011                          Primary Only    
1   country       IN        2011            Primary With Upper Primary    
2   country       IN        2011  Primary With Upper Primary Sec/H.Sec    
3   country       IN        2011                    Upper Primary Only    
4   country       IN        2011         Upper Primary With Sec./H.Sec    

  conditions  year    total  
0       Good  2014  2372586  
1       Good  2014  1765117  
2       Good  2014   492644  
3       Good  2014   471894  
4       Good  2014   236702

In [22]:
list(dataSchoolCom.columns.values)

['Unnamed: 0',
 'YEAR',
 'STATE NAME',
 'DISTRICT NAME',
 'TOTAL POULATION',
 'PERCENTAGE URBAN POPULATION',
 '0-6 POPULATION',
 'GROWTH RATE',
 'SEX RATIO',
 'PERCENTAGE SC POPULATION',
 'PERCENTAGE ST POPULATION',
 'OVERALL LITERACY',
 'FEMALE LITERACY',
 'MALE LITERACY',
 'AREA (SQ. KM) (AREA SQKM)',
 'AGE GROUP 6 TO 10 (TOT 6 10 15)',
 'AGE GROUP 11 TO 13 (TOT 11 13 15)',
 'PRIMARY ONLY (SCH1)',
 'PRIMARY WITH UPPER PRIMARY (SCH2)',
 'PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3)',
 'UPPER PRIMARY ONLY (SCH4)',
 'UPPER PRIMARY WITH SEC./H.SEC (SCH5)',
 'PRIMARY WITH UPPER PRIMARY SEC (SCH6)',
 'UPPER PRIMARY WITH  SEC. (SCH7)',
 'NO RESPONSE (SCH9)',
 'TOTAL (SCHTOT)',
 'PRIMARY ONLY (SCH1G)',
 'PRIMARY WITH UPPER PRIMARY (SCH2G)',
 'PRIMARY WITH UPPER PRIMARY SEC/H.SEC (SCH3G)',
 'UPPER PRIMARY ONLY (SCH4G)',
 'UPPER PRIMARY WITH SEC./H.SEC (SCH5G)',
 'PRIMARY WITH UPPER PRIMARY SEC (SCH6G)',
 'UPPER PRIMARY WITH  SEC. (SCH7G)',
 'NO RESPONSE (SCH9G)',
 'TOTAL (SCHTOTG)',
 'PRIMARY 